In [1]:
from Data_manager.UserUtils import *
from Data_manager.split_functions.split_train_validation_random_holdout import \
    split_train_in_two_percentage_global_sample
URM_all = getURM_all()
URM_train_validation, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train_validation, train_percentage = 0.80)

In [2]:
from Recommenders.MatrixFactorization.Cython.MatrixFactorization_Cython import MatrixFactorization_AsySVD_Cython
model = MatrixFactorization_AsySVD_Cython

from Evaluation.Evaluator import EvaluatorHoldout
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10], ignore_users=[])

EvaluatorHoldout: Ignoring 2579 (20.4%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users


In [3]:
import optuna as op
earlystopping_kwargs =  {"validation_every_n": 5,
                              "stop_on_validation": True,
                              "evaluator_object": evaluator_validation,
                              "lower_validations_allowed": 5,
                              "validation_metric": "MAP",
                              }
def objective(trial):
    # Optuna suggest params
    random_seed = 42
    sgd_mode = trial.suggest_categorical('sgd_mode', ["sgd", "adagrad", "adam"])
    epochs = trial.suggest_categorical('epochs', [500])
    use_bias = trial.suggest_categorical('use_bias', [True, False])
    batch_size = trial.suggest_categorical('batch_size', [1])
    num_factors = trial.suggest_int('num_factors', 1, 200)
    item_reg = trial.suggest_loguniform('item_reg', 1e-5, 1e-2)
    user_reg = trial.suggest_loguniform('user_reg', 1e-5, 1e-2)
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
    negative_interactions_quota = trial.suggest_uniform('negative_interactions_quota', 0.0, 0.5)
    
    recommender = model(URM_train)
    recommender.fit(**trial.params, **earlystopping_kwargs)
    
    result, _ = evaluator_validation.evaluateRecommender(recommender)
    MAP_result = result["MAP"].item()
       
    return MAP_result

In [ ]:
best_params = {
}
study_name = "asysvd-study"  # Unique identifier of the study.
storage_name = "sqlite:///db.db"
study = op.create_study(study_name=study_name, storage=storage_name, direction="maximize", load_if_exists=True)
study.enqueue_trial(best_params)
study.optimize(objective, n_trials=100)

[I 2023-12-07 20:14:35,676] A new study created in RDB with name: asysvd-study
/var/folders/l3/n3yjvx_d2g9dy0ztkyft2th40000gn/T/ipykernel_27649/1697907931.py:16: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  item_reg = trial.suggest_loguniform('item_reg', 1e-5, 1e-2)
/var/folders/l3/n3yjvx_d2g9dy0ztkyft2th40000gn/T/ipykernel_27649/1697907931.py:17: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  user_reg = trial.suggest_loguniform('user_reg', 1e-5, 1e-2)
/var/folders/l3/n3yjvx_d2g9dy0ztkyft2th40000gn/T/ipykernel_27649/1697907931.py:18: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/opt

MatrixFactorization_AsySVD_Cython_Recommender: URM Detected 462 ( 3.7%) users with no interactions.
MatrixFactorization_AsySVD_Cython_Recommender: URM Detected 341 ( 1.5%) items with no interactions.
ASY_SVD: Estimating user factors... 
ASY_SVD: Estimating user factors... done!
ASY_SVD: Processed 306388 (100.0%) in 7.80 sec. MSE loss 6.24E+00. Sample per second: 39221
ASY_SVD: Epoch 1 of 500. Elapsed time 7.06 sec
ASY_SVD: Processed 306388 (100.0%) in 7.27 sec. MSE loss 9.61E+00. Sample per second: 42164
ASY_SVD: Epoch 2 of 500. Elapsed time 13.49 sec
ASY_SVD: Processed 306388 (100.0%) in 6.69 sec. MSE loss 7.63E+00. Sample per second: 45823
ASY_SVD: Epoch 3 of 500. Elapsed time 19.91 sec
ASY_SVD: Processed 306388 (100.0%) in 7.22 sec. MSE loss 5.59E+00. Sample per second: 42417
ASY_SVD: Epoch 4 of 500. Elapsed time 26.45 sec
ASY_SVD: Processed 306388 (100.0%) in 6.61 sec. MSE loss 4.07E+00. Sample per second: 46361
ASY_SVD: Validation begins...
ASY_SVD: Estimating user factors... 
ASY

[I 2023-12-07 20:19:17,739] Trial 0 finished with value: 9.944896538991378e-05 and parameters: {'sgd_mode': 'adam', 'epochs': 500, 'use_bias': True, 'batch_size': 1, 'num_factors': 40, 'item_reg': 3.452803745276082e-05, 'user_reg': 0.002178154860280256, 'learning_rate': 0.02316313754617756, 'negative_interactions_quota': 0.38315886646832575}. Best is trial 0 with value: 9.944896538991378e-05.


MatrixFactorization_AsySVD_Cython_Recommender: URM Detected 462 ( 3.7%) users with no interactions.
MatrixFactorization_AsySVD_Cython_Recommender: URM Detected 341 ( 1.5%) items with no interactions.
ASY_SVD: Estimating user factors... 
ASY_SVD: Estimating user factors... done!


/var/folders/l3/n3yjvx_d2g9dy0ztkyft2th40000gn/T/ipykernel_27649/1697907931.py:16: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  item_reg = trial.suggest_loguniform('item_reg', 1e-5, 1e-2)
/var/folders/l3/n3yjvx_d2g9dy0ztkyft2th40000gn/T/ipykernel_27649/1697907931.py:17: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  user_reg = trial.suggest_loguniform('user_reg', 1e-5, 1e-2)
/var/folders/l3/n3yjvx_d2g9dy0ztkyft2th40000gn/T/ipykernel_27649/1697907931.py:18: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_r

...

ASY_SVD: Estimating user factors... done!
ASY_SVD: Processed 306388 (100.0%) in 16.56 sec. MSE loss 3.82E-01. Sample per second: 18501
ASY_SVD: Epoch 1 of 500. Elapsed time 15.99 sec
ASY_SVD: Processed 306388 (100.0%) in 16.57 sec. MSE loss 3.78E-01. Sample per second: 18493
ASY_SVD: Epoch 2 of 500. Elapsed time 31.99 sec
ASY_SVD: Processed 306388 (100.0%) in 16.60 sec. MSE loss 3.75E-01. Sample per second: 18461
ASY_SVD: Epoch 3 of 500. Elapsed time 48.02 sec
ASY_SVD: Processed 306388 (100.0%) in 16.47 sec. MSE loss 3.73E-01. Sample per second: 18602
ASY_SVD: Epoch 4 of 500. Elapsed time 1.06 min
ASY_SVD: Processed 306388 (100.0%) in 16.53 sec. MSE loss 3.73E-01. Sample per second: 18537
ASY_SVD: Validation begins...
ASY_SVD: Estimating user factors... 
ASY_SVD: Estimating user factors... done!
EvaluatorHoldout: Processed 10059 (100.0%) in 4.00 sec. Users per second: 2515
ASY_SVD: CUTOFF: 10 - PRECISION: 0.0003678, PRECISION_RECALL_MIN_DEN: 0.0005625, RECALL: 0.0004437, MAP: 0.0001124

[I 2023-12-08 01:50:18,649] Trial 9 finished with value: 0.0005861212497060981 and parameters: {'sgd_mode': 'adagrad', 'epochs': 500, 'use_bias': False, 'batch_size': 1, 'num_factors': 172, 'item_reg': 2.401809242457718e-05, 'user_reg': 6.252141368271884e-05, 'learning_rate': 0.00012724049681183785, 'negative_interactions_quota': 0.3693889945361583}. Best is trial 7 with value: 0.014087021493821327.


MatrixFactorization_AsySVD_Cython_Recommender: URM Detected 462 ( 3.7%) users with no interactions.
MatrixFactorization_AsySVD_Cython_Recommender: URM Detected 341 ( 1.5%) items with no interactions.
ASY_SVD: Estimating user factors... 


/var/folders/l3/n3yjvx_d2g9dy0ztkyft2th40000gn/T/ipykernel_27649/1697907931.py:16: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  item_reg = trial.suggest_loguniform('item_reg', 1e-5, 1e-2)
/var/folders/l3/n3yjvx_d2g9dy0ztkyft2th40000gn/T/ipykernel_27649/1697907931.py:17: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  user_reg = trial.suggest_loguniform('user_reg', 1e-5, 1e-2)
/var/folders/l3/n3yjvx_d2g9dy0ztkyft2th40000gn/T/ipykernel_27649/1697907931.py:18: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_r

ASY_SVD: Estimating user factors... done!
ASY_SVD: Processed 306388 (100.0%) in 14.07 sec. MSE loss 3.42E-02. Sample per second: 21768
ASY_SVD: Epoch 1 of 500. Elapsed time 13.22 sec
ASY_SVD: Processed 306388 (100.0%) in 13.36 sec. MSE loss 3.10E-02. Sample per second: 22928
ASY_SVD: Epoch 2 of 500. Elapsed time 26.51 sec
ASY_SVD: Processed 306388 (100.0%) in 13.64 sec. MSE loss 2.90E-02. Sample per second: 22469
ASY_SVD: Epoch 3 of 500. Elapsed time 39.78 sec
ASY_SVD: Processed 306388 (100.0%) in 13.91 sec. MSE loss 2.79E-02. Sample per second: 22032
ASY_SVD: Epoch 4 of 500. Elapsed time 53.05 sec
ASY_SVD: Processed 306388 (100.0%) in 14.06 sec. MSE loss 2.68E-02. Sample per second: 21794
ASY_SVD: Validation begins...
ASY_SVD: Estimating user factors... 
ASY_SVD: Estimating user factors... done!
EvaluatorHoldout: Processed 10059 (100.0%) in 4.00 sec. Users per second: 2516
ASY_SVD: CUTOFF: 10 - PRECISION: 0.0186599, PRECISION_RECALL_MIN_DEN: 0.0407502, RECALL: 0.0369110, MAP: 0.007026

In [11]:
study.best_params

{'num_factors': 69}

In [36]:
final = model(URM_train_validation)
final.fit(**study.best_params)

PureSVDRecommender: URM Detected 217 ( 1.7%) users with no interactions.
PureSVDRecommender: URM Detected 104 ( 0.5%) items with no interactions.
PureSVDRecommender: Computing SVD decomposition...
PureSVDRecommender: Computing SVD decomposition... done in 0.81 sec


In [37]:
from Evaluation.Evaluator import EvaluatorHoldout
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10], ignore_users=[])
evaluator_test.evaluateRecommender(final)

EvaluatorHoldout: Ignoring 2171 (17.2%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Processed 10467 (100.0%) in 4.23 sec. Users per second: 2476


(       PRECISION PRECISION_RECALL_MIN_DEN   RECALL       MAP MAP_MIN_DEN  \
 cutoff                                                                     
 10      0.060055                 0.103973  0.08724  0.030317    0.051596   
 
              MRR      NDCG        F1  HIT_RATE ARHR_ALL_HITS  ...  \
 cutoff                                                        ...   
 10      0.196874  0.097511  0.071139  0.386644      0.242683  ...   
 
        COVERAGE_USER COVERAGE_USER_HIT USERS_IN_GT DIVERSITY_GINI  \
 cutoff                                                              
 10          0.828216          0.320225    0.828216       0.009501   
 
        SHANNON_ENTROPY RATIO_DIVERSITY_HERFINDAHL RATIO_DIVERSITY_GINI  \
 cutoff                                                                   
 10            8.152184                   0.995629             0.027233   
 
        RATIO_SHANNON_ENTROPY RATIO_AVERAGE_POPULARITY RATIO_NOVELTY  
 cutoff                                      

In [ ]:
## MAP = 0.0308313